Как обычно импорты, тут всё стандартно кроме одного: пакета **seaborn**, который нравится лично мне, поэтому я хочу его вам показать.

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn
%matplotlib inline

# Анализ реальных данных

Источник данных: «Российский мониторинг экономического положения и здоровья населения НИУ-ВШЭ (RLMS-HSE)», проводимый Национальным исследовательским университетом  "Высшая школа экономики" и ООО «Демоскоп» при участии Центра народонаселения Университета Северной Каролины в Чапел Хилле и Института социологии РАН. (Сайты обследования RLMS-HSE: http://www.cpc.unc.edu/projects/rlms и http://www.hse.ru/rlms)».

Опрос населения, данные за 2007 год.

In [ ]:
raw_data = pd.read_csv('data.csv', sep=';')

Функция, которая берёт нужный столбец, желающие могут внимательно на неё посмотреть и понять, как она работает :)

In [ ]:
def get_data_by_id(variant, data):
    to_drop = []
    id_variant = 'id' + str(variant)
    for i in range(1, 77):
        if i != variant:
            to_drop.append('id' + str(i))        
    data_id = data.drop(to_drop, axis = 1)
    data_id = data_id[data_id[id_variant] == 1]
    data_id = data_id.drop(id_variant, axis = 1)
    return data_id

Возьмём Алтайский край (id45) в качестве примера

In [ ]:
data_Altai = get_data_by_id(45, raw_data)


Список переменных
1. **public** – 1 если государственное предприятие (в государственной собственности), 0 если частное;

2. **wage** – заработная плата, полученная за последние 30 дней по основному месту работы;

3. **income** – сколько всего денег за последние 30 дней получил респондент, считая все источники дохода;

4. **tenure** – опыт работы на текущем месте;

5. **educ** – уровень образования, категориальная переменная (0 для индивидов без среднего образования)

    1. среднее образование
    2. профессиональные курсы
    3. ПТУ, техническое училище
    4. техникум
    5. институт, университет, академия

6. **urban** – 1 респондент проживает в городе, 0 в сельской местности;
7. **male** – пол, 1 для мужчин, 0 для женщин;
8. **age** – возраст;
9. **subordinates** – наличие подчиненных, 1 если есть, 0 в противном случае.
10. **experience** – число лет общего трудового стажа респондента;
11. **otherjob** – хотелось бы респонденту найти другую работу, 1 если да, 0 в противном случае;
12. **trust** – доверие мелкому и среднему частному бизнесу, 1 если респондент доверяет, 0 если не доверяет;
13. **internet** – приходилось ли респонденту в течение последних 12 месяцев пользоваться Интернетом, 1 если да, 0 в противном случае.

## 1. Первый взгляд на вещи

Что можно сделать в первую очередь? Посмотреть в целом, что за данные у нас есть.

In [ ]:
data_Altai.head()

Теперь посмотрим на основные статистики:

In [ ]:
data_Altai.describe()

## 2. Поиск выбросов

В реальных данных всегда есть выбросы. О том, что такое "выброс" всегда стоит говорить отдельно. Давайте найдём их с помощью крутой штуки под названием **box-plot**.

In [ ]:
plt.figure(figsize=(2,5))
seaborn.boxplot(y='wage', data=data_Altai)

** Задание: ** постройте box-plot по переменным income, age, tenure.

In [ ]:
#YOUR CODE HERE
#just copy it

In [ ]:
#YOUR CODE HERE

In [ ]:
#YOUR CODE HERE

Можно сделать крутую штуку по бинарным переменным: box-plot автоматически поделил данные по классам 0 и 1 и построил график по каждому из значений бинарной переменной.

In [ ]:
plt.figure(figsize=(4,5))
seaborn.boxplot(y='wage', x = 'male', data=data_Altai)

** Задание: ** постройте box-plot по переменным male и income, male и age, urban и wage.

In [ ]:
#YOUR CODE HERE

In [ ]:
#YOUR CODE HERE

In [ ]:
#YOUR CODE HERE

## 3. Гистограммы

Теперь посмотрим на гистограммы:

In [ ]:
data_Altai.wage.hist()

** Задание: ** постройте гистограммы по переменным income, age, tenure.

In [ ]:
#YOUR CODE HERE

In [ ]:
#YOUR CODE HERE

In [ ]:
#YOUR CODE HERE

А теперь красивые картинки кончаются и начинается математика.

## 4. Проверка гипотез

### Проверьте гипотезу, что матожидание среднего дохода равно 9000 рублей против двусторонней альтернативной гипотезы.

Двусторонняя альтернативная гипотеза -- это когда альтернативная гипотеза имеет вид "матожидание среднего дохода не равно 9000".

In [ ]:
income = data_Altai.income

n = income.count() # количество элеметнов в выборке
mean = #YOUR CODE HERE
std = #YOUR CODE HERE

alpha = 0.95 # уровень значимости
mu_0 = 9000 # нулевая гипотеза: матожидание равно 9000 рублей

Сейчас тут будет происходить некоторая математическая магия: (не переживайте, вам не надо будет её прогать! всё в порядке!) мы не знаем точную дисперсию, поэтому нам понадобится t-распределение Стьюдента, которое находится в библиотеке scipy модуле stats. t-распределение Сьюдента кроме уровня значимости имеет также степень свободы, который равен n-1. 

Критическая статистика равна $$t = \frac{\overline{X} - \mu_0}{S / \sqrt{n}}$$

где $\overline{X}$ -- среднее, $S$ -- стандартное отклонение, $n$ -- количество элементов в выборке, $\mu_0$ -- наше предположение.

Теперь мы должны посмотреть, находится ли она в критической области, где мы *не отвергаем* гипотезу.

In [ ]:
from scipy import stats

t = (mean - mu_0) / (std / n ** 0.5)

x = np.linspace(-3,3,1000)
d = stats.t.fit(income)
plt.plot(x, stats.t.pdf(x, n-1), linewidth=3.3, label='t-Student')
left, right = stats.t.interval(alpha, df=n-1)
plt.vlines([left, right], 0.0, stats.t.pdf(tcrit, n-1), colors='r', linewidth=3, label='критическая область')
fill = np.arange(left,right,0.01)
plt.fill_between(fill, stats.t.pdf(fill, n-1), color='r', alpha=0.5)
plt.scatter(t, 0, marker='o', c='k', label='критическая точка')
plt.legend(bbox_to_anchor=(0.6, 0.85),
           bbox_transform=plt.gcf().transFigure)
plt.show()

## 5. Рассчёт p-value

Рассчитаем p-value для предыдущей задачи. Для этого проделаем "обратную операцию". Если бы мы делали это вручную, то посмотрели бы по табличке, какому значению p-value соотвествует наше критическое значение. В питоне же есть функция stats.t.sf(t, df), которая сделает это за нас.

In [ ]:
print('p-value:', 2*min(1 - stats.t.cdf(t, n-1), stats.t.cdf(t, n-1)))